In [ ]:
#  This Notebook does not work in it's current form; mostly LDBC files that have moved ..
#
#  Code to harvest below; reading a CSV and outputting Parquet.


#  CONNECTION HANDLE

import os

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

In [ ]:
# ##################################################################

In [ ]:
#  LOAD FROM CSV

#  The data files are not in place, fyi ..

l_inp = "gs://farrell-bucket/ldbc/50_csv_sf-0.003-bi-node_headers.txt"
l_enp = "gs://farrell-bucket/ldbc/51_csv_sf-0.003-bi-edge_headers.txt"

# l_dir = "gs://katana-demo-datasets/csv-datasets/ldbc"
l_dir = "gs://farrell-bucket/ldbc"


import_data.csv(
   my_graph,
      #
   input_node_path     = l_inp,
   input_edge_path     = l_enp,
   input_dir           = l_dir,
      #
   data_delimiter      = "|",
   schema_delimiter    = "|",
   files_have_headers  = True,
   )


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
# ##################################################################

In [ ]:
#  VARIABLES ONLY

import dask.dataframe as dd
import numpy as np
import os
   #
from google.cloud import storage
   #
import pandas as pd
import gcsfs


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/work/03_MyKeys.json"

l_Bucket     = "farrell-bucket"

l_InputDir  = "sf-0.003-csv"                       #  These values should not start with a slash
l_OutputDir = "sf-0.003-parquet"


print("--")


In [ ]:
# ##################################################################

In [ ]:
#  READ CSV, OUTPUT PARQUET

def f_ReadCsvOutputParquet(i_Bucket, i_File, i_OutputDir):

   l_File = "gs://" + i_Bucket + "/" + i_File
      #
   # print("Processing: " + i_File)
   print(".", end="")

   df = dd.read_csv(l_File,
      delimiter='|',
      dtype={
      "id": np.dtype("O"),
      "name": np.dtype("O"),
      "url": np.dtype("O"),
         #
      # "creationDate": np.dtype("O"),
      "locationIP": np.dtype("O"),
      "browserUsed": np.dtype("O"),
      "content": np.dtype("O"),
      "length": np.dtype("O"),
         #
      "imageFile": np.dtype("O"),
      "language": np.dtype("O"),
         #
      "title": np.dtype("O"),
         #
      "firstName": np.dtype("O"),
      "lastName": np.dtype("O"),
      "gender": np.dtype("O"),
      # "birthday": np.dtype("O"),
      "speaks": np.dtype("O"),
      "email": np.dtype("O"),
         #
      "START_ID": np.dtype("O"),
      "END_ID": np.dtype("O"),
      },

      converters={
         'creationDate': lambda x: pd.to_datetime(x, unit='ns'),
         'birthday': lambda x: pd.to_datetime(x).date(),
            # 
         #'language': lambda x: x.split(';'),
         #'email': lambda x: x.split(';')
      }
      )

   # print(df.count())
   # print(df.head(1))

   i_TrueFile1 = os.path.basename(i_File)
   i_TrueFile2 = os. path. splitext(i_TrueFile1)[0]
      #
   l_OutputDir = "gs://" + i_Bucket + "/" + i_OutputDir + "/" + i_TrueFile2

   df.to_parquet(
      l_OutputDir,
      write_index=False,
      write_metadata_file=False,
      engine="pyarrow",
      version="2.6")

print("--")

In [ ]:
#  GCS doesn't have directories like a standard filesystem
#
#  This loop will output top level folder names ending with a "/",
#  and all child files as a relative pathname

def f_ProcessFiles(i_Bucket, i_InputDir, i_OutputDir):
    
   l_files = storage.Client().get_bucket(i_Bucket)

   for l_file in l_files.list_blobs(prefix=i_InputDir):
      if (l_file.name.endswith(".csv")):
         f_ReadCsvOutputParquet(i_Bucket, l_file.name, i_OutputDir)

print("--")

In [ ]:
# Driver for function above

print("   ", end="")

f_ProcessFiles(l_Bucket, l_InputDir, l_OutputDir)

print(""     )
print("   --")


In [ ]:
# ##################################################################

In [ ]:
#  Better versions of the 2 above; allows for skipping, and quantity

In [ ]:
#  GCS doesn't have directories like a standard filesystem
#
#  This loop will output top level folder names ending with a "/",
#  and all child files as a relative pathname

def f_ProcessFiles(i_Bucket, i_InputDir, i_OutputDir, i_start, i_end):
    
   l_files = storage.Client().get_bucket(i_Bucket)
      #
   l_cntr = 0

   for l_file in l_files.list_blobs(prefix=i_InputDir):
      if (l_file.name.endswith(".csv")):
         l_cntr = l_cntr + 1
            #
         if (l_cntr >= i_start) and (l_cntr <= i_end):
            f_ReadCsvOutputParquet(i_Bucket, l_file.name, i_OutputDir)

print("--")


In [ ]:
#  Driver for function above
#
#  These ranges are inclusive

l_start = 3
l_end   = 5

print("   ", end="")

f_ProcessFiles(l_Bucket, l_InputDir, l_OutputDir, l_start, l_end)

print(""     )
print("   --")
